In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import json
import os
from functions import colorize, complex_correlation, fidelity

# for Jupyter lab
%matplotlib widget
# for Jupyter notebook
# %matplotlib notebook

data_folder = '../Data'

SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title

In [2]:
## use fidelity or correlation to compare matrices

function = fidelity
# function = complex_correlation

# 1. First load the params and data

## Load parameters

In [3]:
path = os.path.sep.join([data_folder,'param.json'])
with open(path,'r') as f:
    param = json.load(f)
    print('Succesfully loaded parameters from .json file')

Succesfully loaded parameters from .json file


## Load theoretical modes

In [28]:
path = os.path.sep.join([data_folder,'conversion_matrices.npz'])

data_dict = np.load(path)
# list of the radial momenta of the modes
L = data_dict['L_modes']
# list of the angular orbital momebta of the modes
M = data_dict['M_modes']
modes_in = data_dict.f.modes_in
# create mode matrix for two polarizations
modes_in_full = np.kron(np.array([[1,0],[0,1]]),modes_in)
# number of input pixels
m = np.int(np.sqrt(modes_in.shape[1]))
modes_out = data_dict.f.modes_out
modes_out_full = np.kron(np.array([[1,0],[0,1]]),modes_out)
# number of output pixels
n = np.int(np.sqrt(modes_out.shape[1]))

## Load the TMs

In [29]:
TM_modes = []
for i, pos in enumerate(param['Translation']['steps_pressure']):
    path = os.path.sep.join([data_folder,f'TM_modes_{str(pos)}.npz'])
    data_dict = np.load(path)
    TM_modes.append(data_dict.f.TM_corr_modes)
TM_modes = np.array(TM_modes)
Nmodes = TM_modes.shape[-1]//2

In [30]:
# Index corresponding to Delta x = 0
index_dx0 = 17
# Index used for the reference matrix
index_reference = 5
# Index of the deformation used for the maximum deformation considered
index_perturbed = -1

TM_perturb = TM_modes[index_perturbed]
TM_ref = TM_modes[index_reference]
TM_correlation = []

# Vector of deformation values
dx = (np.array(param['Translation']['steps_pressure'])-param['Translation']['steps_pressure'][index_dx0])*1e3

# Caclulate the fidelty/correlation of the matrix compared to the reference one
for i,TM in enumerate(TM_modes):
    TM_correlation.append(function(TM_modes[index_reference],TM))
    
plt.figure()
plt.plot(dx[index_reference:],TM_correlation[index_reference:],'r')
plt.plot([dx[index_perturbed]],[TM_correlation[index_perturbed]],'b+', markersize=16)
plt.plot([dx[index_reference]],[TM_correlation[index_reference]],'g+', markersize=16)
plt.legend(['Fidelity','Maximum perturbation','Reference'])
plt.xlabel(r'Postion ($\mu m$)')
plt.ylabel('Complex correlation')
plt.title(r'Decorrelation as a function of displacement')

# Save
# plt.savefig('fidelityTM.svg', dpi=400)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Decorrelation as a function of displacement')

# 2. Decorrelation of the TM

In [33]:
%matplotlib widget

index = -1
index_dx0 = 17
index_reference = 5
TM_perturb = TM_modes[index]
TM_ref = TM_modes[index_reference]
TM_correlation = []

dx = (np.array(param['Translation']['steps_pressure'])-param['Translation']['steps_pressure'][index_dx0])*1e3
for i,TM in enumerate(TM_modes):
    TM_correlation.append(function(TM_modes[index_reference],TM))
plt.figure(),
plt.plot(dx[index_reference:],TM_correlation[index_reference:],'r')
plt.plot([dx[index]],[TM_correlation[index]],'b+', markersize=16)
plt.plot([dx[index_reference]],[TM_correlation[index_reference]],'g+', markersize=16)
plt.legend(['corr','index_perturb','index_reference'])
plt.xlabel(r'Postion ($\mu m$)')
plt.ylabel('Complex correlation')

# save figure
#plt.savefig('corrTM.svg', dpi=400)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Complex correlation')

# 3. Channels robust to deformation
Deformation principal modes obtained from the eigenmodes of the Generalized Wigner-Smith operator 
(Fig.3 of the main text)

## Estimate the Wigner-Smith operator

In [34]:
def get_WS(H0,H1,H2):
    '''
    Get the Wigner-Smith operator from 3 matrices at Dx-dx, Dx and Dx+dx
    '''
    dH = (H2 - H0)*0.5
    WS = -1j*np.linalg.pinv(H1)@dH
    return WS

def get_WS_symm(H0,H1,H2):
    '''
    Get the Hermitian part of the Wigner-Smith operator from 3 matrices at Dx-dx, Dx and Dx+dx
    '''
    WS = get_WS(H0,H1,H2)
    WS_symm = WS + WS.T.conj()
    return WS_symm


# central deformation to estimate the WS operator
center_TM_index = 24
# step to evaluate the derivative
step = 4

WS_symm = get_WS_symm(TM_modes[center_TM_index - step], 
                      TM_modes[center_TM_index], 
                      TM_modes[center_TM_index + step])

## Eigenvalue decomposition of the Wigner-Smith operator

In [35]:
s,U = np.linalg.eig(WS_symm)
# sort the eigenvalues
s = s[np.argsort(s)]
U = U[np.argsort(s),:]

## Analyze the (de)correlation of the deformation principal modes
compared to the fundamental mode and to random input wavefronts

In [36]:
def get_out_states(in_modes, i0, i1):
    '''
    Get the output states corresponding to the input state 'in_modes'
    for the deformations indexed by 'i0' and 'i1'.
    Return the output fields for the two polarizations for each deformation. 
    '''
    output_state = TM_modes[i0] @ in_modes
    out_pix = modes_out_full.transpose()@output_state
    out_pix_p1_0 = out_pix[:n**2].reshape([n]*2)
    out_pix_p2_0 = out_pix[n**2:].reshape([n]*2)

    output_state = TM_modes[i1] @ in_modes
    out_pix = modes_out_full.transpose()@output_state
    out_pix_p1_f = out_pix[:n**2].reshape([n]*2)
    out_pix_p2_f = out_pix[n**2:].reshape([n]*2)
    
    return out_pix_p1_0, out_pix_p2_0, out_pix_p1_f, out_pix_p2_f

def get_correlation_out(in_modes, indexes, index_reference):
    '''
    Take as argument an input wavefront 'in_modes' (in the mode basis), 
    a list of indexes 'indexes' and the reference index 'index_reference'.
    Returns a list of values representing the 
    correlation of the output intensity  
    for each deformation indexed by 'indexes' 
    with the output intensity for the reference deformation 
    indexed by 'index_reference'
    '''
    corr_vector = []
    f = lambda x: np.abs(x)**2
    for ind in indexes:
        out_pix_p1_0, out_pix_p2_0, out_pix_p1_f, out_pix_p2_f = get_out_states(in_modes, i0 = index_reference, i1 = ind)
        out_0 = np.concatenate([out_pix_p1_0,out_pix_p2_0])
        out_f = np.concatenate([out_pix_p1_f,out_pix_p2_f])
        corr_vector.append(complex_correlation(f(out_0),f(out_f)))
    return corr_vector
        

In [37]:
Nmodes = TM_modes[0].shape[0]
# compare to the deformation around witch the WS operator is estimated.
index_reference = center_TM_index
# number of average random wavefront to average on
n_ave = 20


## DEFORMATION PRINCIPAL MODES
indexes = list(range(len(TM_modes)))
correlations_WS = []
for ind in range(Nmodes):
    state_WS = U[:,ind] # use WS eigen vector
    correlations_WS.append(get_correlation_out(state_WS, 
                                               indexes = indexes, 
                                               index_reference = index_reference))
## FUNDAMENTAL MODE
correlations_mode = []
state = np.zeros(Nmodes, dtype = np.complex) # use the modes 
# send the fundamental mode in one polarization
state[55] = 1
correlations_mode = get_correlation_state(state, indexes = indexes, index_reference = index_reference)

## RANDOM WAVEFRONTS
correlations_rand_avg = np.zeros(len(TM_modes))
for j in range(n_ave):
    # generate a random combination of mode as input
    rand_vec = (np.random.rand(Nmodes)-0.5) + 1j * (np.random.rand(Nmodes)-0.5)
    output_states_rand = [TM_modes[i] @ rand_vec for i in range(len(TM_modes))]
    correlations_rand = np.array([complex_correlation(output_states_rand[i],output_states_rand[index_reference]) for i in range(len(TM_modes))])
    correlations_rand_avg += correlations_rand
correlations_rand_avg /= n_ave

NameError: name 'get_correlation_state' is not defined

## Plot output intensity correlation data 

In [ ]:
from matplotlib import colors

def get_color_map(n):
    cdict = {'red':   ((0.0,  0.3, 0.3),
                       (1.0,  1.0, 1.0)),

             'green': ((0.0,  0.1, 0.1),
                       (1.,  0., 0.)),

             'blue':  ((0.0,  0.0, 0.0),
                       (1.0,  0.0, 0.0))}
    cmap = colors.LinearSegmentedColormap('custom', cdict, N=n)
    return cmap

In [ ]:
i0 = 15
index_dx0 = 17
plt.close('all')
plt.figure()
dx = (np.array(param['Translation']['steps_pressure'])-param['Translation']['steps_pressure'][index_dx0])*1e3
cmap = get_color_map(n = Nmodes)

for i,corr_WS in enumerate(correlations_WS):
    plt.plot(dx[i0:],corr_WS[i0:],color = cmap(i))
    
for i,corr_WS in enumerate(correlations_mode):
    plt.plot(dx[i0:],correlations_mode[i][i0:], color = 'darkgreen', marker='+')

plt.plot(dx[i0:],correlations_rand_avg[i0:],'b+-')
all_min = plt.gca().get_ylim()[0]
plt.vlines(dx[center_TM_index],all_min,1., 'r', linestyles = 'dashed')

plt.legend(['Fondamental mode', 'Random wavefronts', 'Deformation principal modes'])
plt.xlabel('Deformation in microns')
plt.ylabel('Correlation')

# save figure
# plt.savefig('WS_corr.svg')

## Display output intensity patterns

### Fundamental mode

In [38]:
in_state = np.zeros(Nmodes) # use the modes 
in_state[0] = 1

# to_img = colorize
to_img = lambda x: np.abs(x)**2
cmap = 'gray'

out_pix_p1_0, out_pix_p2_0, out_pix_p1_f, out_pix_p2_f = get_out_states(in_state,
                                                                       i0 = index_dx0,
                                                                       i1 = -1)
out_0 = to_img(np.concatenate([out_pix_p1_0,out_pix_p2_0]))
out_f = to_img(np.concatenate([out_pix_p1_f,out_pix_p2_f]))
corr = complex_correlation(out_0,out_f)

plt.figure()
plt.subplot(221)
plt.imshow(to_img(out_pix_p1_0), cmap = cmap)
plt.title('Dx = 0')
plt.axis('off')
plt.subplot(223)
plt.imshow(to_img(out_pix_p2_0), cmap = cmap)
plt.axis('off')

plt.subplot(222)
plt.imshow(to_img(out_pix_p1_f), cmap = cmap)
plt.title('Dx = max')
plt.axis('off')
plt.subplot(224)
plt.imshow(to_img(out_pix_p2_f), cmap = cmap)
plt.axis('off')

plt.suptitle(f'Pearson correlation coefficient = {corr:.2f}%')

# save figure
# plt.savefig('fundamental_mode.svg')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'Pearson correlation coefficient = 0.58%')

### Random wavefront

In [39]:
rand_vec = (np.random.rand(Nmodes)-0.5) + 1j * (np.random.rand(Nmodes)-0.5)

out_pix_p1_0, out_pix_p2_0, out_pix_p1_f, out_pix_p2_f = get_out_states(rand_vec,
                                                                       i0 = index_dx0,
                                                                       i1 = -1)
out_0 = to_img(np.concatenate([out_pix_p1_0,out_pix_p2_0]))
out_f = to_img(np.concatenate([out_pix_p1_f,out_pix_p2_f]))
corr = complex_correlation(out_0,out_f)

plt.figure()
plt.subplot(221)
plt.imshow(to_img(out_pix_p1_0), cmap = cmap)
plt.title('Dx = 0')
plt.axis('off')
plt.subplot(223)
plt.imshow(to_img(out_pix_p2_0), cmap = cmap)
plt.axis('off')

plt.subplot(222)
plt.imshow(to_img(out_pix_p1_f), cmap = cmap)
plt.title('Dx = max')
plt.axis('off')
plt.subplot(224)
plt.imshow(to_img(out_pix_p2_f), cmap = cmap)
plt.axis('off')

plt.suptitle(f'Pearson correlation coefficient = {corr:.2f}%')

# save figure
# plt.savefig('rand.svg')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'Pearson correlation coefficient = 0.48%')

### First deformation principal modes

In [40]:
max_ind = np.argmax([c[-1] for c in correlations_WS])
cmap = 'gray'
to_img = lambda x: np.abs(x)**2

for ind in list(range(6))+[max_ind]:
    state_WS = U[:,ind] # use WS eigen vector
    out_pix_p1_0, out_pix_p2_0, out_pix_p1_f, out_pix_p2_f =  get_out_states(state_WS,
                                                                   i0 = index_dx0,
                                                                   i1 = -1)
    
    max_1 = np.max([np.max(to_img(out_pix_p1_0)),np.max(to_img(out_pix_p1_f))])
    max_2 = np.max([np.max(to_img(out_pix_p2_0)),np.max(to_img(out_pix_p2_f))])
    
    plt.figure()
    plt.subplot(221)
    plt.imshow(to_img(out_pix_p1_0), vmax = max_1, cmap = cmap)
    plt.title('Input')
    plt.axis('off')
    plt.subplot(223)
    plt.imshow(to_img(out_pix_p2_0), vmax = max_2, cmap = cmap)
    plt.axis('off')
    
    plt.subplot(222)
    plt.imshow(to_img(out_pix_p1_f), vmax = max_1, cmap = cmap)
    plt.title('Output')
    plt.axis('off')
    plt.subplot(224)
    plt.imshow(to_img(out_pix_p2_f), vmax = max_2, cmap = cmap)
    plt.axis('off')
    
    plt.suptitle(rf'Corr for $\Delta x$ max = {correlations_WS[ind][-1]:.2f}')
    
    # save figure
    # plt.savefig(f'WS_mode_{ind}.svg')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# 4. PCA of the deformation operator as a function of $\Delta x$ 
(Fig.4 of the main text)

### Calculation of the deformation operator
$D(t) = H^{-1}(\Delta z = 0)\times H(\Delta z) -I$

In [41]:
#
index_min_svd = index_dx0
index_max_svd = 52
# index of the reference matrix
index_reference = index_min_svd

evolutionTMs = np.array([TM@np.linalg.pinv(TM_modes[index_reference])-np.eye(TM.shape[0]) for TM in TM_modes])
evolution_vec = evolutionTMs.reshape([evolutionTMs.shape[0],-1]).transpose()

## Singular valude decomposition

In [42]:
u,s,v = np.linalg.svd(evolution_vec[:,index_min_svd:index_max_svd],full_matrices = False)

n = len(s)
xticks = list(range(0,n,5))
yticks = list(np.arange(0,1.1,0.2))


fig, ax = plt.subplots(1,1, figsize = (6*.8,4*.8))
img = ax.plot(s/s[0],'+', markersize = 11)
ax.set_xticks(xticks)
# ax.set_xlim([0,25])
ax.set_xticklabels(xticks)
ax.set_yticks(yticks)
ax.set_yticklabels([f'{i:.1f}' for i in yticks])
plt.title('Singular values.');

# save figure
# plt.savefig('svdD.svg', dpi = 300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Singular value distribution  

In [43]:
dist_bins = np.linspace(0,1,30)
plt.figure(figsize = (6*.8,4*.8))
plt.hist(s/s[0], bins = dist_bins, density=True);

# save figure
# plt.savefig('svdD_dist.svg', dpi = 300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
n_max = Nmodes

cmap = 'gray'

# to_img = lambda x: colorize(x, beta = 1.8)
to_img = lambda x: np.abs(x)**2

U = u.reshape([Nmodes]*2+[u.shape[-1]])
fig, ax = plt.subplots(2,2)
plt.subplots_adjust(wspace = 0.1, hspace = 0.0, top = 1., bottom = 0)
fig.set_size_inches(8, 8)
ax[0,0].imshow(to_img(U[:n_max,:n_max,0]),cmap)
ax[0,0].axis('off')
ax[0,0].set_title(fr'$U_0$ (sv = {s[0]/s[0]:.2f})')
ax[0,1].imshow(to_img(U[:n_max,:n_max,1]),cmap, vmax = np.max(to_img(U[:n_max,:n_max,2]))*0.5)
# ax[0,1].imshow(to_img(U[:n_max,:n_max,1]),cmap)
ax[0,1].axis('off')
ax[0,1].set_title(fr'$U_1$ (sv = {s[1]/s[0]:.2f})')
ax[1,0].imshow(to_img(U[:n_max,:n_max,2]),cmap)
ax[1,0].axis('off')
ax[1,0].set_title(fr'$U_2$ (sv = {s[2]/s[0]:.2f})')
ax[1,1].imshow(to_img(U[:n_max,:n_max,3]),cmap)
ax[1,1].axis('off')
ax[1,1].set_title(fr'$U_3$ (sv = {s[3]/s[0]:.2f})')

# save figure
# plt.savefig('Ui.svg',dpi = 400)

print(np.sum(s[:2]**2)/np.sum(s**2))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.9794973604143481


## Show the map of close-by mode matrix
Map of the modes with 1 if the modes have a difference of angular and radial momentum below 1, 0 otherwise

In [50]:
fig ,ax = plt.subplots(1,1)
plt.subplots_adjust(wspace = 0.1, top = 1., bottom = 0)

ML = (np.abs(M) < 2)*(np.abs(L) < 2).astype(np.complex)
ax.imshow( colorize(ML),'jet', vmax = 1.1)
ax.set_title(r'$|\Delta l|+|\Delta m|$')
ax.axis('off');

# save figure
# plt.savefig('Deltam_Deltal.svg',dpi = 400)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Approximate TM using the first singular components

In [57]:
def correlation_with_approx(TM0, TM1, U, alpha, n_sv = 2):
    '''
    Calculates the correlation between the measured matrix and the one 
    estimated using only the first 'n_sv' singular components with weights 'alpha'.
    'TM0' is the reference matrix and 'TM1' the one we want to approximate.
    'U' are the singular vectors.
    '''
    new_TM = np.zeros_like(TM0)
    for i in range(n_sv):
        new_TM += alpha[i] * U[...,i]

    new_TM = np.copy(TM0)+new_TM@np.copy(TM0)
    corr = function(TM1, new_TM)

    return corr

In [58]:
index_min_to_show = 10
# number of singular vector/values to keep in the estimation
n_sv = 2

v2 = np.diag(1./s)@u.transpose().conjugate()@evolution_vec[:,index_min_to_show:]

def get_reduced_sv_TM(index, n_sv, display = False):
    '''
    Approximate the evolution operator with the 'n_sv' first singular vectors
    and return the correlation between the true 'index'-th matrix and 
    the estimated one.
    '''
    SVh = (np.diag(s)@v2[:,index])
    alphas = SVh[:n_sv]
    return correlation_with_approx( TM_modes[index_min_svd], 
                          TM_modes[index_min_to_show+index], 
                          U, 
                          alpha = alphas, 
                          n_sv = n_sv) 

## Plot the correlation between approximate matrices and true ones

In [60]:
mask = np.zeros_like(s)
mask[:n_sv] = 1.
T0 = TM_modes[index_reference,...]
evolution_vec_estim = (u*(s*mask))@v2
estimated_TMs = [T0+T@T0 for T in evolution_vec_estim.reshape([Nmodes]*2+[-1]).transpose(2,0,1)]
real_TMs = TM_modes[index_min_to_show:,...]
energy = [np.linalg.norm(A)**2/np.linalg.norm(B)**2 for A,B in zip(estimated_TMs,real_TMs)]
    
n_sv_vec = [n_sv]
corr_reduced_sv2 = [[get_reduced_sv_TM(i,n) for i in range(v2.shape[1])] for n in n_sv_vec] 

fig = plt.figure(figsize = (2.5*3,1*3))
ax1 = fig.add_subplot(111)
ax1.plot(dx[index_min_to_show:],corr_reduced_sv2[0])
ax1.vlines(dx[index_min_svd],0,1., 'r', linestyles = 'dashed')

y1min,y1max = 0.8, 1.03
ylocs1 = np.arange(y1min,y1max,0.05)
ax1.set_yticks(ylocs1)
ax1.set_yticklabels([rf'{i*100:g}%' for i in ylocs1], fontsize = 15)
ax1.set_ylim([y1min,y1max])
# xlocs = [f'{i:g}' for i in ax1.get_xticks()]
xticks = np.arange(0,70,10)
ax1.set_xticks(xticks)
ax1.set_xticklabels(xticks, fontsize = 15)
ax1.set_xlim([-8,66])

ax2 = plt.gca().twinx()
ax2.plot(dx[index_min_to_show:],energy, color = 'orange')
y2min,y2max = 0.8, 1.03
ylocs2 = np.arange(y2min,y2max,0.05)
ax2.set_ylim([y2min,y2max])
ax2.set_yticks(ylocs2)
ax2.set_yticklabels([rf'{i*100:g}%' for i in ylocs2], fontsize = 15)


energy_tot = np.linalg.norm(evolution_vec_estim)**2/np.linalg.norm(evolution_vec)**2
plt.suptitle(f'Energy kept = {energy_tot*100:.2f}%')

# save figure
# plt.savefig('SV_approx.svg')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'Energy kept = 96.26%')